In [ ]:
import torch
import time
import math
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence
import gzip
import csv

In [ ]:
USE_GPU = True
device = torch.device('mps' if USE_GPU else 'cpu')

In [ ]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
dataset_sample=  [
    ['Maclean', 'English'],
    ['Vajnichy', 'Russion'],
    ['Nasikovsky', 'Russion'],
    ['Usami', 'Japanese'],
    ['Fionin', 'Russion'],
    ['Sharkey', 'English'],
    ['Bakagul', 'Russion'],
    ['Pakhrin', 'Russion'],
    ['Tansho', 'Japanese']
]

In [ ]:
BATCH_SIZE = 256
HIDDEN_SIZE = 100
N_LAYER = 2
N_EPOCHS = 10
N_CHARS = 128

In [ ]:
class NameDataSet(Dataset):
    def __init__(self, is_train_set=True):
        filename = 'data/names_train.csv.gz' if is_train_set else 'data/names_test.csv.gz'
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f)
            rows = list(reader)
        self.names = [row[0] for row in rows]
        self.len = len(self.names)
        self.countries = [row[1] for row in rows]
        self.country_list = list(sorted(set(self.countries)))
        self.country_dict = self.getCountryDict()
        self.country_num = len(self.country_list)
    def getCountryDict(self):
        country_dict= dict()
        for idx, country_name in enumerate(self.country_list, 0):
            country_dict[country_name] = idx
        return country_dict
    def idx2country(self, index):
        return self.country_list[index]
    def getCountriesNum(self):
        return self.country_num
    def __getitem__(self, index):
        return self.names[index], self.country_dict[self.countries[index]]
    def __len__(self):
        return self.len

In [ ]:
trainset = NameDataSet(True)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = NameDataSet(False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

N_COUNTRY = trainset.getCountriesNum()

In [ ]:
batch_size = 1
seq_len = 5
input_size = 3
hidden_size = 4
n_layers = 2
bidirectional = True

gru = torch.nn.GRU(input_size, hidden_size, n_layers, bidirectional=bidirectional)

input = torch.rand(15).reshape(seq_len, batch_size, input_size)
print('input shape: (seq_len, batch_size, input_size)', input.shape)
init_hidden = torch.zeros(n_layers*2 if bidirectional else n_layers, batch_size, hidden_size)
print('init_hidden shape: (n_layers*2, batch_size, hidden_size)', init_hidden.shape)

outputs, hidden = gru(input, init_hidden)
print('outputs shape: (seq_len, batch_size, hidden_size*2)', outputs.shape)
print('hidden shape: (n_layers*2, batch_size, hidden_size)', hidden.shape)

In [ ]:
class RNNClassfier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassfier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = 2 if bidirectional else 1

        self.embeding = torch.nn.Embedding(input_size, hidden_size)
        self.gru = torch.nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)
        self.fc = torch.nn.Linear(hidden_size * self.n_directions, output_size)
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_size)
        return hidden.to(device) if USE_GPU else hidden
    
    def forward(self, input, seq_len):
        batch_size = input.size(0)
        input = input.t()
        hidden = self._init_hidden(batch_size)
        embedding = self.embeding(input)
        padded_input = pack_padded_sequence(embedding, seq_len.cpu())
        output, hidden = self.gru(padded_input, hidden)

        if(self.n_directions == 2):
            hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim=1)
        else:
            hidden_cat = hidden[-1]
        fc_output = self.fc(hidden_cat)
        return fc_output

classifier = RNNClassfier(N_CHARS, HIDDEN_SIZE, N_COUNTRY, N_LAYER)
if(USE_GPU):
  classifier = classifier.to(device)

In [ ]:
def create_tensor(tensor):
    if USE_GPU:
        tensor = tensor.to(device)
    return tensor

def name2list(name):
     arr = [ord(c) for c in name]
     return arr, len(arr)

def make_tensors(names, countries):
    sequences_and_lengths = [name2list(name) for name in names]
    name_sequences = [sl[0] for sl in sequences_and_lengths]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequences_and_lengths])
    countries = countries.long()
   
    # make tensor of name, BatchSize x SeqLen
    seq_tensor = torch.zeros(len(name_sequences), seq_lengths.max()).long()
    for idx, (seq, seq_len) in enumerate(zip(name_sequences, seq_lengths), 0):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    # sort by length to use pack_padded_sequence
    seq_lengths, perm_idx = seq_lengths.sort(dim=0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    countries = countries[perm_idx]
    return create_tensor(seq_tensor), create_tensor(seq_lengths), create_tensor(countries)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

In [ ]:
def train(epoch, start):
    total_loss = 0
    for i, (names, countries) in enumerate(trainloader, 1):
        inputs, seq_len, target = make_tensors(names, countries)
        output = classifier(inputs, seq_len)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if i % 10 == 0:
            print(f'[{time_since(start)}] Epoch {epoch} ', end='')
            print(f'[{i * len(inputs)}/{len(trainset)}] ', end='')
            print(f'loss={total_loss / (i * len(inputs))}')
    return total_loss

In [ ]:
def test():
    correct = 0
    total = len(testset)
    print("evaluating trained model ...") 
    with torch.no_grad():
        for i, (names, countries) in enumerate(testloader, 1):
            inputs, seq_len, target = make_tensors(names, countries)
            output = classifier(inputs, seq_len)
            pred = output.max(dim=1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        percent = '%.2f' % (100 * correct / total)
        print(f'Test set: Accuracy {correct}/{total} {percent}%')
    return correct / total

In [ ]:
def train_loop():
    start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    acc_list = []

    for epoch in range(1, N_EPOCHS + 1):
        train(epoch, start)
        acc = test()
        acc_list.append(acc)

 

In [ ]:
train_loop()